In [1]:
from LandBase import *

def biz_worker_to_db(con, p_yyyy='2015') :
    ####
    # "KOSIS" -> 온라인간행물 -> 전국사업체조사 -> 읍면동별 산업대분류별 현황
    # 조회가능기간: 2000 ~ 2015
    # 테이블: biz_worker
    ####
    
    # DataFrame 작성
    columns = ['admd_cd', 'admd_nm', 'biz_nm', \
               'tot_company_cnt', 'tot_worker_cnt', \
               'male_worker_cnt', 'female_worker_cnt', \
               'personal_company_cnt', 'personal_worker_cnt', \
               'corp_company_cnt', 'corp_worker_cnt', \
               'non_corp_company_cnt', 'non_corp_worker_cnt', \
               'grp_company_cnt', 'grp_worker_cnt', \
               'p_1_4_company_cnt', 'p_1_4_worker_cnt', 'p_5_9_company_cnt', 'p_5_9_worker_cnt', \
               'p_10_19_company_cnt', 'p_10_19_worker_cnt', 'p_20_49_company_cnt', 'p_20_49_worker_cnt', \
               'p_50_99_company_cnt', 'p_50_99_worker_cnt', 'p_100_299_company_cnt', 'p_100_299_worker_cnt', \
               'p_300_499_company_cnt', 'p_300_499_worker_cnt', 'p_500_999_company_cnt', 'p_500_999_worker_cnt', \
               'p_1000_company_cnt', 'p_1000_worker_cnt']
    df = pd.read_excel('./source_files/kosis_온라인간행물_전국사업체조사_읍면동별산업대분류_' + p_yyyy +'.xlsx', \
                       skiprows=[0,1, 2], parse_cols=32, names=columns, header=None, encoding='euc-kr')
    df.columns = columns
    df['admd_cd'] = df['admd_cd'].str.extract('(\d+)', expand=False).astype(int)
    df = df[ (df['admd_cd'].astype(str).str.startswith('11')) |\
            (df['admd_cd'].astype(str).str.startswith('23')) |\
            (df['admd_cd'].astype(str).str.startswith('31'))]
    df.replace(np.NaN, 0, inplace=True)
    df['yyyy'] = p_yyyy

    # DB처리
    table_name = 'biz_worker'
    sql = 'delete from ' + table_name + ' where yyyy = \'' + p_yyyy + '\'';
    con.execute(sql)
    df.to_sql(name=table_name, con=con, if_exists='append', index=False, chunksize=1000)


In [2]:
# DB Connection Open
con = get_db_connection()

# "KOSIS"의 읍면동별 산업대분류별 현황
for yyyy in range(2015, 2016, 1) :
    print(yyyy)
    biz_worker_to_db(con, str(yyyy))

# DB Connection Close
disconnect_db(con)

print('Program Finised')

2015
Program Finised
